In [14]:
cd somdst/son-dst3/

/opt/ml/somdst/son-dst3


In [4]:
!nvidia-smi

Wed May 19 07:03:40 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:00:05.0 Off |                  Off |
| N/A   40C    P0    71W / 250W |   2762MiB / 32480MiB |     39%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [15]:
!python train_somdst.py --model_name_or_path BonjinKim/dst_kor_bert

2021-05-19 11:22:57.318807: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2021-05-19 11:22:57.318851: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/opt/ml/somdst/son-dst3
wandb: Currently logged in as: bongjinkim (use `wandb login --relogin` to force relogin)
2021-05-19 11:23:00.680145: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2021-05-19 11:23:00.680191: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up 

KeyboardInterrupt: 

In [17]:
!python train_somdst.py --model_name_or_path dsksd/bert-ko-small-minimal --model_dir /opt/ml/model --learning_rate 0.000009 --model_name coco_dst --ckpt 41

2021-05-20 08:07:33.641573: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2021-05-20 08:07:33.641628: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/opt/ml/somdst/son-dst3
wandb: Currently logged in as: bongjinkim (use `wandb login --relogin` to force relogin)
2021-05-20 08:07:37.134353: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2021-05-20 08:07:37.134475: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up 

In [21]:
!python train_trade.py

2021-05-20 16:31:10.382376: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2021-05-20 16:31:10.382430: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Traceback (most recent call last):
  File "train_trade.py", line 77, in <module>
    train_data, dev_data, dev_labels = load_dataset(train_data_file)
  File "/opt/ml/somdst/son-dst3/data_utils.py", line 50, in load_dataset
    data += json.load(open(d, 'rt', encoding='UTF8'))
IsADirectoryError: [Errno 21] Is a directory: '/'


In [17]:
import os
os.path.exists(os.path.join("/opt/ml/input/data/train_dataset", f"train_somdst_features_bert.pkl"))

False

In [20]:
import json
a= json.load(open('/opt/ml/input/data/train_dataset/new_train.json'))
b=json.load(open('/opt/ml/input/data/train_dataset/train_dials.json'))

In [21]:
len(a)

7000

In [23]:
type(b)

list

In [11]:
from difflib import SequenceMatcher
import json
from tqdm.notebook import tqdm
import nltk
from nltk.translate import bleu
from nltk.translate.bleu_score import SmoothingFunction
smoothie = SmoothingFunction().method4

def similar(a, b, only_bleu=False, with_bleu=False):
    sim = SequenceMatcher(None, a, b).ratio()
    if not only_bleu and not with_bleu:
        return sim
    bleu_score = bleu(a, b)
    if only_bleu:
        return bleu_score
    return (sim + bleu_score) / 2

with open("/opt/predictions/cleaning_1th.csv", "rb") as f:
    a = json.load(f)
cnt = 0
all_cnt = 0
for key, values in a.items():
    for i, value in enumerate(values):
        if "시간" in value:
            continue
        if ":" in value or "=" in value or "&" in value or "(" in value or ")" in value :
            print(f" before : {value}", end="\t")

        if "=" in value:
            a[key][i] = value.replace(" = ", "=")

        if "&" in value:
            a[key][i] = value.replace(" & ", "&")
        if "(" in value:
            value = value.replace("( ", "(")
            value = value.replace(" ) ", ")")
            value = value.replace(" )", ")")
            if value[-1] == "역":
                value = value.replace(" (", "(")
            a[key][i] = value
        if "동대문" in value and value[-1] == "(":
            value = "관광-이름-동대문 (흥인지문)"
            a[key][i] = value
        if "월드컵 경기장(" in value:
            a[key][i] = "관광-이름-서울 월드컵 경기장 (상암 월드컵 경기장)"
            
        if ":" in value or "=" in value or "&" in value or "(" in value or ")" in value:
            print(f" after : {a[key][i]}")
            cnt+=1
        
        all_cnt+=1
print(f"{cnt} / {all_cnt}")

json.dump(
    a,
    open(f"post-cleaning.csv", "w"),
    indent=2,
    ensure_ascii=False,
)



with open("post-cleaning.csv", "rb") as f:
    predctions = json.load(f)
with open("ontology.json", "rb") as f:
    ontology = json.load(f)

all_values = sorted(set(sum(ontology.values(), [])))
name_values = sorted(set(ontology['관광-이름'] + ontology['식당-이름'] + ontology['숙소-이름'] + ontology['택시-도착지'] + ontology['택시-출발지']))
a = ontology['관광-이름'].copy()
b = ontology['숙소-이름'].copy()
c = ontology['식당-이름'].copy()
ontology['관광-이름'] = [n for n in name_values if n not in b + c + ontology['지하철-도착지']]
ontology['숙소-이름'] = [n for n in name_values if n not in a + c + ontology['지하철-도착지']]
ontology['식당-이름'] = [n for n in name_values if n not in a + b + ontology['지하철-도착지']]


cnt = 0
for guid, states in tqdm(predctions.items()):
    flag = False
    for i, state in enumerate(states):
        dom, slot, value = state.split('-')
        domslot = f"{dom}-{slot}"
        if "시간" in slot:
            continue
        if value in all_values:
            continue
        if ('게스트' in value or '호텔' in value or '모텔' in value or '하우스' in value) and '그린' not in value:
            value = value.split(' ')[0]
        if domslot in ['택시-출발지', '택시-도착지']:
            values = all_values
        else:
            values = ontology[domslot]

        similarities = sorted([(v, similar(value, v, with_bleu=True)) for v in values] ,reverse=True, key=lambda x: x[1])
        print(domslot, value, "\t=>\t", similarities[:3])

        predctions[guid][i] = f"{domslot}-{similarities[0][0]}"
        flag = True
    if flag:
        cnt+=1
print(f"{cnt} / 14771")
json.dump(
    predctions,
    open(f"post-cleaning.csv", "w"),
    indent=2,
    ensure_ascii=False,
)

FileNotFoundError: [Errno 2] No such file or directory: '/opt/ml/model/coco_dst/model-30.bin'